In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [108]:
df = pd.read_csv("./credit_train.csv")
df = df.drop("Unnamed: 0",axis=1)
df = df.drop("Unnamed: 0.1",axis=1)
# 定义特征和目标
X = df.drop("credit_level", axis=1)
X = X.drop("uid", axis=1)
y = df["credit_level"]

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14)

In [111]:
rf = RandomForestClassifier(n_estimators=600, random_state=14,max_depth=80)

# 训练数据
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=80, n_estimators=600, random_state=14)

In [112]:
# 预测测试集
y_pred = rf.predict(X_test)

# 计算精度
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.7879383808587348


In [113]:
df

,uid,credit_level,COUNT(dm_v_tr_contract_mx),MAX(dm_v_tr_contract_mx.buss_amt),MAX(dm_v_tr_contract_mx.term_mth),MEAN(dm_v_tr_contract_mx.buss_amt),MEAN(dm_v_tr_contract_mx.term_mth),MIN(dm_v_tr_contract_mx.buss_amt),MIN(dm_v_tr_contract_mx.term_mth),MODE(dm_v_tr_contract_mx.apply_type),...,SUM(dm_v_tr_gzdf_mx. is_secu_card),SUM(dm_v_tr_gzdf_mx. tran_amt),COUNT(dm_v_tr_sbyb_mx),MODE(dm_v_tr_sbyb_mx. tran_sts),NUM_UNIQUE(dm_v_tr_sbyb_mx. tran_sts),COUNT(pri_cust_liab_info),MAX(pri_cust_liab_info.all_bal),MEAN(pri_cust_liab_info.all_bal),MIN(pri_cust_liab_info.all_bal),SUM(pri_cust_liab_info.all_bal)
0,320511197112153312,60,-0.582244,1.618251,3.430277,2.220947,3.284679,1.992257,3.622202,1.885848,...,-0.531118,-0.439538,3.817438,-0.25887,-0.261501,2.179201,2.336421,2.595417,2.726543,3.120927
1,320825197205134342,60,-0.582244,-0.735411,-0.331844,-0.669947,-0.117317,-0.471569,0.189630,-0.420824,...,-0.531118,-0.439538,-0.464234,-0.25887,-0.261501,2.032990,-1.322279,-1.237120,-1.312149,0.008800
2,320911196205095606,60,-0.582244,-0.687931,-0.357970,-0.611629,-0.157817,-0.401744,0.148273,-0.420824,...,-0.531118,-0.439538,-0.464234,-0.25887,-0.261501,-0.686540,-0.112559,-0.192058,-0.215765,-0.515526
3,320911196309012174,85,1.549489,-0.416615,-0.331844,-0.655665,-0.580174,-0.900494,-1.092415,-0.420824,...,1.767510,0.384454,0.106655,-0.25887,3.824078,-0.686540,-0.112559,-0.192058,-0.215765,-0.515526
4,320911196312192495,60,-0.582244,-0.823589,-0.331844,-0.778251,-0.117317,-0.601244,0.189630,-0.420824,...,-0.531118,-0.439538,-0.464234,-0.25887,-0.261501,-0.686540,-0.112559,-0.192058,-0.215765,-0.515526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30502,654123198611223715,60,-0.582244,-0.077471,-0.331844,0.138171,-0.137567,0.496007,0.148273,-0.420824,...,-0.531118,-0.439538,-0.464234,-0.25887,-0.261501,-0.686540,-0.112559,-0.192058,-0.215765,-0.515526
30503,654123199102253734,60,2.828528,0.600818,-0.331844,-0.791068,-0.126664,-0.920444,0.148273,1.885848,...,-0.531118,-0.439538,-0.464234,-0.25887,-0.261501,-0.686540,-0.112559,-0.192058,-0.215765,-0.515526
30504,654323197108173972,85,-0.155897,-0.552273,-0.331844,-0.445007,-0.144317,-0.202244,0.106917,1.885848,...,-0.531118,-0.439538,-0.464234,-0.25887,-0.261501,-0.686540,-0.112559,-0.192058,-0.215765,-0.515526
30505,654324198004269794,60,-0.155897,0.939963,-0.331844,1.387836,-0.117317,1.992257,0.189630,1.885848,...,-0.531118,-0.439538,-0.464234,-0.25887,-0.261501,1.272691,1.758142,2.150453,2.643551,2.153051


In [114]:
from joblib import dump
dump(rf, './model/random_forest_model_credit.joblib')

['./model/random_forest_model_credit.joblib']

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("./credit_train.csv")
df = df.drop("Unnamed: 0",axis=1)
df = df.drop("Unnamed: 0.1",axis=1)
# 定义特征和目标
X = df.drop("credit_level", axis=1)
X = X.drop("uid", axis=1)
# y = df["credit_level"]/5
# y = y.astype(int)
# print(y.unique())
# print(y)
y = df["credit_level"]
le = LabelEncoder()
y = le.fit_transform(y)
for i, j in enumerate(le.classes_):
    print('{} --> {}'.format(j,i))

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14)

35 --> 0
50 --> 1
60 --> 2
70 --> 3
85 --> 4


In [3]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 5,
    'max_depth': 6,
    'eval_metric': 'mlogloss',
    'learning_rate': 0.1, 
    'min_child_weight': 3, 
    'seed': 0,         
    'subsample': 1, 
    'colsample_bytree': 0.8, 
    'gamma': 0.01,
    'reg_alpha': 0.1, 
    'reg_lambda': 1
}


In [4]:
dtrain = xgb.DMatrix(X_train,label = y_train)
dtest = xgb.DMatrix(X_test,label = y_test)

In [5]:
num_rounds = 100
model = xgb.train(params,dtrain,num_rounds,evals=[(dtest,'test')])

[0]	test-mlogloss:1.45442
[1]	test-mlogloss:1.33169
[2]	test-mlogloss:1.23095
[3]	test-mlogloss:1.14602
[4]	test-mlogloss:1.07349
[5]	test-mlogloss:1.01141
[6]	test-mlogloss:0.95713
[7]	test-mlogloss:0.90868
[8]	test-mlogloss:0.86558
[9]	test-mlogloss:0.82710
[10]	test-mlogloss:0.79353
[11]	test-mlogloss:0.76382
[12]	test-mlogloss:0.73629
[13]	test-mlogloss:0.71179
[14]	test-mlogloss:0.68926
[15]	test-mlogloss:0.66915
[16]	test-mlogloss:0.65086
[17]	test-mlogloss:0.63486
[18]	test-mlogloss:0.61992
[19]	test-mlogloss:0.60609
[20]	test-mlogloss:0.59385
[21]	test-mlogloss:0.58281
[22]	test-mlogloss:0.57241
[23]	test-mlogloss:0.56300
[24]	test-mlogloss:0.55465
[25]	test-mlogloss:0.54681
[26]	test-mlogloss:0.53978
[27]	test-mlogloss:0.53344
[28]	test-mlogloss:0.52773
[29]	test-mlogloss:0.52264
[30]	test-mlogloss:0.51776
[31]	test-mlogloss:0.51333
[32]	test-mlogloss:0.50906
[33]	test-mlogloss:0.50535
[34]	test-mlogloss:0.50198
[35]	test-mlogloss:0.49866
[36]	test-mlogloss:0.49567
[37]	test-m

In [6]:
# 预测测试集
y_pred = model.predict(dtest)
print(y_pred)
# 计算精度
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

[2. 2. 2. ... 3. 2. 3.]
Accuracy: 0.7990822681088168


In [41]:
model.save_model('./model/xgboost_model_credit.bin')